In [ ]:
"""
QUY TRÌNH XỬ LÝ DỮ LIỆU - PART 2
==================================
Data Cleaning & Advanced Preprocessing
"""


from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from part_2 import DataExplorationPipeline


# ============================================================================
# TIẾP TỤC CLASS DataExplorationPipeline
# ============================================================================


class DataCleaningPipeline(DataExplorationPipeline):
    """
    Mở rộng class với các chức năng cleaning và preprocessing
    """

    def __init__(self, data_path=None, df=None):
        super().__init__(data_path, df)
        self.df_cleaned = None
        self.imputation_strategies = {}
        self.scaling_strategies = {}
        self.encoding_strategies = {}

    

    # ========================================================================
    # 1. XỬ LÝ GIÁ TRỊ BỊ THIẾU (MISSING VALUES HANDLING)
    # ========================================================================

    def handle_missing_values(self, strategies=None, threshold=0.7):
        """
        Xử lý giá trị bị thiếu với nhiều chiến lược
        
        Parameters:
        -----------
        strategies : dict
            Dictionary chứa chiến lược cho từng cột
            Ví dụ: {'column_name': 'mean', 'other_col': 'knn'}
        threshold : float
            Ngưỡng để xóa cột (nếu tỷ lệ thiếu > threshold)
        
        Strategies available:
        - 'drop': Xóa hàng
        - 'mean': Điền trung bình
        - 'median': Điền trung vị
        - 'mode': Điền chế độ (mode)
        - 'constant': Điền giá trị cố định
        - 'ffill': Forward fill
        - 'bfill': Backward fill
        - 'knn': KNN Imputer
        - 'iterative': MICE/Iterative Imputer
        """

        print("\n" + "="*80)
        print("🔧 XỬ LÝ GIÁ TRỊ BỊ THIẾU")
        print("="*80)


        self.df_cleaned = self.df.copy()


        # Bước 1: Xóa các cột có quá nhiều missing
        missing_pct = self.df_cleaned.isnull().sum() / len(self.df_cleaned)
        cols_to_drop = missing_pct[missing_pct > threshold].index.tolist()

        if cols_to_drop:
            print(f"\n🗑️  Xóa {len(cols_to_drop)} cột có >70% giá trị thiếu:")
            for col in cols_to_drop:
                print(f"  • {col}: {missing_pct[col]*100:.1f}% thiếu")

            self.df_cleaned.drop(columns=cols_to_drop, inplace=True)



        # Bước 2: Áp dụng strategies
        if strategies is None:
            strategies = self._auto_imputation_strategy()

        print("\n📋 Strategies được áp dụng:")

        for col, strategy in strategies.items():
            if col not in self.df_cleaned.columns:
                continue

            missing_count = self.df_cleaned[col].isnull().sum()
            if missing_count == 0:
                continue

            print(f"\n  → {col} ({missing_count} giá trị thiếu):")
            print(f"     Strategy: {strategy}")

            try:
                if strategy == 'drop':
                    self.df_cleaned.dropna(subset=[col], inplace=True)
                
                elif strategy == 'mean':
                    self.df_cleaned[col].fillna(self.df_cleaned[col].mean(), inplace=True)
                
                elif strategy == 'median':
                    self.df_cleaned[col].fillna(self.df_cleaned[col].median(), inplace=True)
                
                elif strategy == 'mode':
                    mode_val = self.df_cleaned[col].mode()
                    if len(mode_val) > 0:
                        self.df_cleaned[col].fillna(mode_val[0], inplace=True)
                
                elif strategy == 'constant':
                    fill_val = 0 if pd.api.types.is_numeric_dtype(self.df_cleaned[col]) else 'Unknown'
                    self.df_cleaned[col].fillna(fill_val, inplace=True)
                
                elif strategy == 'ffill':
                    self.df_cleaned[col].fillna(method='ffill', inplace=True)
                
                elif strategy == 'bfill':
                    self.df_cleaned[col].fillna(method='bfill', inplace=True)
                
                elif strategy == 'knn':
                    self._apply_knn_imputer([col])
                
                elif strategy == 'iterative':
                    self._apply_iterative_imputer([col])
                
                self.imputation_strategies[col] = strategy
                print(f"     ✓ Hoàn thành")

            except Exception as e:
                print(f"     ✗ Lỗi: {str(e)}")

        # Báo cáo kết quả
        remaining_missing = self.df_cleaned.isnull().sum().sum()
        print(f"\n✅ Kết quả:")
        print(f"  • Giá trị thiếu còn lại: {remaining_missing}")
        print(f"  • Số hàng: {len(self.df)} → {len(self.df_cleaned)}")
        print(f"  • Số cột: {len(self.df.columns)} → {len(self.df_cleaned.columns)}")
        

        return self.df_cleaned
    


    def _auto_imputation_strategy(self):
        """Tự động chọn strategy phù hợp cho từng cột"""

        strategies = {}

        for col in self.df.columns:
            if self.df[col].isnull().sum() == 0:
                continue

            # Numeric columns
            if col in self.numeric_cols:
                # Nếu phân phối gần normal → mean, ngược lại → median
                skewness = abs(self.df[col].skew())
                strategies[col] = 'median' if skewness > 1 else 'mean'

            # Categorical columns
            elif col in self.categorical_cols:
                strategies[col] = 'mode'

            # Datetime columns
            elif col in self.datetime_cols:
                strategies[col] = 'ffill'

        
        return strategies
    


    def _apply_knn_imputer(self, columns, n_neighbors=5):
        """Áp dụng KNN Imputer cho các cột số"""

        numeric_cols = [c for c in columns if c in self.numeric_cols]
        if not numeric_cols:
            return
        
        imputer = KNNImputer(n_neighbors=n_neighbors)
        self.df_cleaned[numeric_cols] = imputer.fit_transform(self.df_cleaned[numeric_cols])



    def _apply_iterative_imputer(self, columns, max_iter=10):
        """Áp dụng Iterative Imputer (MICE)"""

        numeric_cols = [c for c in columns if c in self.numeric_cols]
        if not numeric_cols:
            return
        
        imputer = IterativeImputer(max_iter=max_iter, random_state=42)
        self.df_cleaned[numeric_cols] = imputer.fit_transform(self.df_cleaned[numeric_cols])



    # ========================================================================
    # 2. PHÁT HIỆN VÀ XỬ LÝ NGOẠI LAI (OUTLIER DETECTION & HANDLING)
    # ========================================================================
    

    def detect_outliers(self, methods=['iqr', 'zscore'], visualize=True):
        """
        Phát hiện outliers bằng nhiều phương pháp
        
        Parameters:
        -----------
        methods : list
            Các phương pháp: 'iqr', 'zscore', 'isolation_forest'
        visualize : bool
            Có vẽ box plot hay không
        """
        print("\n" + "="*80)
        print("🔍 PHÁT HIỆN NGOẠI LAI (OUTLIERS)")
        print("="*80)

        if self.df_cleaned is None:
            self.df_cleaned = self.df.copy()

        outlier_info = {}

        for col in self.numeric_cols:
            if col not in self.df_cleaned.columns:
                continue

            outliers = {}
            data = self.df_cleaned[col].dropna()

            # Phương pháp IQR
            if 'iqr' in methods:
                Q1 = data.quantile(0.25)
                Q3 = data.quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                
                iqr_outliers = ((data < lower_bound) | (data > upper_bound)).sum()
                outliers['iqr'] = {
                    'count': iqr_outliers,
                    'percentage': iqr_outliers / len(data) * 100,
                    'lower_bound': lower_bound,
                    'upper_bound': upper_bound
                }

            # Phương pháp Z-Score
            if 'zscore' in methods:
                z_scores = np.abs(stats.zscore(data))
                zscore_outliers = (z_scores > 3).sum()
                outliers['zscore'] = {
                    'count': zscore_outliers,
                    'percentage': zscore_outliers / len(data) * 100
                }
            
            outlier_info[col] = outliers


        # In báo cáo
        print("\n📊 Báo cáo Outliers:")
        for col, methods_result in outlier_info.items():
            print(f"\n  {col}:")
            for method, result in methods_result.items():
                print(f"    • {method.upper()}: {result['count']} outliers ({result['percentage']:.2f}%)")
                if 'lower_bound' in result:
                    print(f"      Range: [{result['lower_bound']:.2f}, {result['upper_bound']:.2f}]")
        

        # Visualization
        if visualize and len(self.numeric_cols) > 0:if visualize and len(self.numeric_cols) > 0:
            cols_to_plot = [c for c in self.numeric_cols[:12] if c in self.df_cleaned.columns]
            n_cols = 3
            n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols
        

            fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4*n_rows))
            axes = axes.flatten() if n_rows > 1 else [axes]
        

            for idx, col in enumerate(cols_to_plot):
                self.df_cleaned.boxplot(column=col, ax=axes[idx])
                axes[idx].set_title(f'{col}', fontsize=10)
                axes[idx].set_ylabel('')
                axes[idx].grid(True, alpha=0.3)

            
            for idx in range(len(cols_to_plot), len(axes)):
                axes[idx].axis('off')

            plt.suptitle('Box Plots - Phát hiện Outliers', fontsize=14, fontweight='bold', y=1.00)
            plt.tight_layout()
            plt.show()
        
        self.report['outliers'] = outlier_info
        return outlier_info
    

    def handle_outliers(self, strategy='cap', columns=None, iqr_multiplier=1.5):
        """
        Xử lý outliers
        
        Parameters:
        -----------
        strategy : str
            'cap': Giới hạn (Winsorization)
            'remove': Loại bỏ
            'log': Biến đổi log
            'boxcox': Biến đổi Box-Cox
        columns : list
            Danh sách cột cần xử lý (None = tất cả numeric)
        iqr_multiplier : float
            Hệ số nhân với IQR
        """

        print("\n" + "="*80)
        print("🔧 XỬ LÝ NGOẠI LAI")
        print("="*80)

        if self.df_cleaned is None:
            self.df_cleaned = self.df.copy()

        if columns is None:
            columns = self.numeric_cols

        columns = [c for c in columns if c in self.df_cleaned.columns]

        print(f"\nStrategy: {strategy}")
        print(f"Số cột xử lý: {len(columns)}\n")

        for col in columns:
            data = self.df_cleaned[col].copy()
            original_count = len(data)

            if strategy == 'cap':
                # Winsorization
                Q1 = data.quantile(0.25)
                Q3 = data.quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - iqr_multiplier * IQR
                upper_bound = Q3 + iqr_multiplier * IQR

                # Cap values
                capped = self.df_cleaned[col].clip(lower=lower_bound, upper=upper_bound)
                changed = (self.df_cleaned[col] != capped).sum()
                self.df_cleaned[col] = capped

                print(f"  • {col}: {changed} giá trị bị cap")

            elif strategy == 'remove':
                # Remove outliers

                Q1 = data.quantile(0.25)
                Q3 = data.quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - iqr_multiplier * IQR
                upper_bound = Q3 + iqr_multiplier * IQR
                
                mask = (data >= lower_bound) & (data <= upper_bound)
                self.df_cleaned = self.df_cleaned[mask]
                removed = original_count - len(self.df_cleaned)
                
                print(f"  • {col}: {removed} hàng bị loại bỏ")


            elif strategy == 'log':
                # Log transformation
                if (data > 0).all():
                    self.df_cleaned[col] = np.log1p(data)
                    print(f"  • {col}: Đã áp dụng log transform")
                else:
                    print(f"  • {col}: Bỏ qua (có giá trị <= 0)")


            elif strategy == 'boxcox':
                # Box-Cox transformation
                if (data > 0).all():
                    transformed, lambda_param = stats.boxcox(data)
                    self.df_cleaned[col] = transformed
                    print(f"  • {col}: Đã áp dụng Box-Cox (λ={lambda_param:.3f})")
                else:
                    print(f"  • {col}: Bỏ qua (có giá trị <= 0)")

        print(f"\n✅ Kết quả: {len(self.df)} → {len(self.df_cleaned)} hàng")
        return self.df_cleaned
    


    # ========================================================================
    # 3. XỬ LÝ DỮ LIỆU PHÂN LOẠI (CATEGORICAL ENCODING)
    # ========================================================================

    def encode_categorical(self, encoding_map=None, max_categories=10):
        """
        Encode các biến phân loại
        
        Parameters:
        -----------
        encoding_map : dict
            Dictionary chỉ định phương pháp encode cho từng cột
            Ví dụ: {'col1': 'onehot', 'col2': 'label', 'col3': 'ordinal'}
        max_categories : int
            Số categories tối đa cho one-hot encoding
        """

        print("\n" + "="*80)
        print("🔤 ENCODING DỮ LIỆU PHÂN LOẠI")
        print("="*80)

        if self.df_cleaned is None:
            self.df_cleaned = self.df.copy()

        if encoding_map is None:
            encoding_map = self._auto_encoding_strategy(max_categories)

        print("\n📋 Strategies:")

        for col, method in encoding_map.items():
            if col not in self.df_cleaned.columns:
                continue
            
            n_unique = self.df_cleaned[col].nunique()
            print(f"\n  → {col} ({n_unique} categories):")
            print(f"     Method: {method}")

            try:
                if method == 'label':
                    # Label Encoding
                    le = LabelEncoder()
                    self.df_cleaned[col] = le.fit_transform(self.df_cleaned[col].astype(str))
                    self.encoding_strategies[col] = {'method': 'label', 'encoder': le}
                
                elif method == 'onehot':
                    # One-Hot Encoding
                    dummies = pd.get_dummies(self.df_cleaned[col], prefix=col, drop_first=True)
                    self.df_cleaned = pd.concat([self.df_cleaned, dummies], axis=1)
                    self.df_cleaned.drop(columns=[col], inplace=True)
                    self.encoding_strategies[col] = {'method': 'onehot', 'columns': dummies.columns.tolist()}
                    print(f"     ✓ Tạo {len(dummies.columns)} cột mới")

                elif method == 'ordinal':
                    # Ordinal Encoding (cần có thứ tự)
                    # User phải cung cấp thứ tự
                    print(f"     ⚠️  Cần cung cấp thứ tự cho ordinal encoding")

                elif method == 'frequency':
                    # Frequency Encoding
                    freq_map = self.df_cleaned[col].value_counts(normalize=True).to_dict()
                    self.df_cleaned[col + '_freq'] = self.df_cleaned[col].map(freq_map)
                    self.encoding_strategies[col] = {'method': 'frequency', 'map': freq_map}
                    print(f"     ✓ Tạo cột {col}_freq")

                print(f"     ✓ Hoàn thành")

            except Exception as e:
                print(f"     ✗ Lỗi: {str(e)}")

        print(f"\n✅ Shape sau encoding: {self.df_cleaned.shape}")
        return self.df_cleaned


    def _auto_encoding_strategy(self, max_categories):
        """Tự động chọn phương pháp encoding"""
        strategies = {}
        
        for col in self.categorical_cols:
            if col not in self.df.columns:
                continue
            
            n_unique = self.df[col].nunique()
            
            if n_unique == 2:
                strategies[col] = 'label'  # Binary → Label
            elif n_unique <= max_categories:
                strategies[col] = 'onehot'  # Few categories → One-Hot
            else:
                strategies[col] = 'frequency'  # Many categories → Frequency
        
        return strategies


    # ========================================================================
    # 4. CHUẨN HÓA DỮ LIỆU (FEATURE SCALING)
    # ========================================================================

    def scale_features(self, method='standard', columns=None):
        """
        Chuẩn hóa các features số
        
        Parameters:
        -----------
        method : str
            'standard': StandardScaler (z-score)
            'minmax': MinMaxScaler (0-1)
            'robust': RobustScaler (sử dụng median, robust với outliers)
        columns : list
            Danh sách cột cần scale (None = tất cả numeric)
        """

        print("\n" + "="*80)
        print("📏 CHUẨN HÓA DỮ LIỆU")
        print("="*80)

        if self.df_cleaned is None:
            self.df_cleaned = self.df.copy()
        
        # Xác định các cột số
        if columns is None:
            columns = [c for c in self.df_cleaned.columns 
                       if pd.api.types.is_numeric_dtype(self.df_cleaned[c])]
            
        columns = [c for c in columns if c in self.df_cleaned.columns]

        print(f"\nMethod: {method}")
        print(f"Số cột: {len(columns)}")

        # Chọn scaler
        if method == 'standard':
            scaler = StandardScaler()
        elif method == 'minmax':
            scaler = MinMaxScaler()
        elif method == 'robust':
            scaler = RobustScaler()
        else:
            raise ValueError(f"Method không hợp lệ: {method}")
        

        # Áp dụng scaling
        self.df_cleaned[columns] = scaler.fit_transform(self.df_cleaned[columns])
        self.scaling_strategies = {'method': method, 'columns': columns, 'scaler': scaler}
        

        print(f"\n✅ Đã chuẩn hóa {len(columns)} cột bằng {method}")

        # Hiển thị thống kê sau scaling
        print("\n📊 Thống kê sau scaling (5 cột đầu):")
        print(self.df_cleaned[columns[:5]].describe())
        
        return self.df_cleaned


    # ========================================================================
    # 5. XỬ LÝ DỮ LIỆU TRÙNG LẶP VÀ KHÔNG NHẤT QUÁN
    # ========================================================================
    
    def clean_data_inconsistencies(self):
        """
        Xử lý dữ liệu trùng lặp và không nhất quán
        """

        print("\n" + "="*80)
        print("🧹 LÀM SẠCH DỮ LIỆU KHÔNG NHẤT QUÁN")
        print("="*80)

        if self.df_cleaned is None:
            self.df_cleaned = self.df.copy()

        # 1. Xử lý trùng lặp
        duplicates_before = self.df_cleaned.duplicated().sum()
        print(f"\n🔄 Dữ liệu trùng lặp:")
        print(f"  • Trước: {duplicates_before} hàng")

        self.df_cleaned.drop_duplicates(inplace=True)

        duplicates_after = self.df_cleaned.duplicated().sum()
        print(f"  • Sau: {duplicates_after} hàng")
        print(f"  • Đã loại bỏ: {duplicates_before - duplicates_after} hàng")

        # 2. Chuẩn hóa string columns
        print(f"\n📝 Chuẩn hóa cột text:")
        string_cols = [c for c in self.categorical_cols if c in self.df_cleaned.columns]
        
        for col in string_cols:
            if self.df_cleaned[col].dtype == 'object':
                # Strip whitespace
                self.df_cleaned[col] = self.df_cleaned[col].astype(str).str.strip()

                # Lowercase (tùy chọn)
                # self.df_cleaned[col] = self.df_cleaned[col].str.lower()
                
                # Thay thế multiple spaces
                self.df_cleaned[col] = self.df_cleaned[col].str.replace(r'\s+', ' ', regex=True)
                
                print(f"  ✓ {col}")

        
        print(f"\n✅ Shape sau cleaning: {self.df_cleaned.shape}")
        return self.df_cleaned

    # ========================================================================
    # 6. TẠO BÁO CÁO TỔNG KẾT
    # ========================================================================

    def generate_cleaning_report(self):
        """
        Tạo báo cáo tổng kết quá trình cleaning
        """
        print("\n" + "="*80)
        print("📊 BÁO CÁO TỔNG KẾT QUÁ TRÌNH CLEANING")
        print("="*80)
        
        print("\n🔢 Thay đổi về kích thước:")
        print(f"  • Hàng: {len(self.df):,} → {len(self.df_cleaned):,} "
              f"({(len(self.df_cleaned)-len(self.df))/len(self.df)*100:+.1f}%)")
        print(f"  • Cột: {len(self.df.columns)} → {len(self.df_cleaned.columns)} "
              f"({len(self.df_cleaned.columns)-len(self.df.columns):+d})")
        
        print("\n🕳️  Missing values:")
        missing_before = self.df.isnull().sum().sum()
        missing_after = self.df_cleaned.isnull().sum().sum()
        print(f"  • Trước: {missing_before:,}")
        print(f"  • Sau: {missing_after:,}")
        print(f"  • Giảm: {missing_before - missing_after:,}")
        
        print("\n🔧 Strategies đã áp dụng:")
        print(f"  • Imputation: {len(self.imputation_strategies)} cột")
        print(f"  • Encoding: {len(self.encoding_strategies)} cột")
        if self.scaling_strategies:
            print(f"  • Scaling: {len(self.scaling_strategies.get('columns', []))} cột")
        
        # Lưu cleaned data
        print("\n💾 Lưu dữ liệu đã xử lý:")
        print("  df_cleaned = pipeline.df_cleaned")
        
        return self.df_cleaned


# ============================================================================
# HƯỚNG DẪN SỬ DỤNG PART 2
# ============================================================================

"""
CÁCH SỬ DỤNG:

# Khởi tạo (tiếp tục từ Part 1)
pipeline = DataCleaningPipeline(df=your_dataframe)

# 1. Xử lý missing values
pipeline.handle_missing_values(
    strategies={'column1': 'mean', 'column2': 'mode'},
    threshold=0.7
)

# 2. Phát hiện và xử lý outliers
pipeline.detect_outliers(methods=['iqr', 'zscore'], visualize=True)
pipeline.handle_outliers(strategy='cap', iqr_multiplier=1.5)

# 3. Encode categorical variables
pipeline.encode_categorical(max_categories=10)

# 4. Scale features
pipeline.scale_features(method='standard')

# 5. Clean inconsistencies
pipeline.clean_data_inconsistencies()

# 6. Báo cáo tổng kết
cleaned_df = pipeline.generate_cleaning_report()

# 7. Lưu kết quả
cleaned_df.to_csv('cleaned_data.csv', index=False)
"""

print("\n✅ ĐÃ HOÀN THÀNH PART 2: DATA CLEANING & PREPROCESSING")
print("📌 Tiếp theo: Part 3 - Relationship Analysis & Feature Selection")
        
